# Imports

In [1]:
import os
import jsonlines
from tqdm.notebook import tqdm

import lamini
import pandas as pd

from contract_poc.utils import read_pdf, build_prompts_from_dataframe, load_format_pages
from contract_poc.qa_pipeline import QuestionAnswerPipeline, load_qa_prompts, save_answers
from contract_poc.answer_pipeline import AnswerPipeline
from contract_poc.rag_pipeline import RAGPipeline
from contract_poc.gen_pipeline import GenPipeline, simple_prompt_generator, save_answers_to_csv
from contract_poc.summary_pipeline import SummaryPipeline, save_summaries

# Data Wrangling

In [2]:
uber_text = read_pdf("data/uber_2021.pdf")
print(f"Uber number of pages: {len(uber_text)}")

Uber number of pages: 307


In [3]:
lyft_text = read_pdf("data/lyft_2021.pdf")
print(f"Lyft number of pages: {len(lyft_text)}")

Lyft number of pages: 238


In [4]:
eval_data = []
with jsonlines.Reader(open(os.path.join("eval_sets","gold-test-set.jsonlines"), "rb")) as reader:
    for line in reader:
        eval_data.append(line)

In [5]:
eval_data[:4]

[{'question': 'What countries does Uber operate in?',
  'answer': 'Our technology is available in approximately 72 countries around the world, principally in the United States (“U.S.”) and Canada, Latin America, Europe, the Middle East, Africa, and Asia (excluding China and Southeast Asia).'},
 {'question': 'What countries does Lyft operate in?',
  'answer': 'United states and select cities in Canada'},
 {'question': 'Do Uber and Lyft operate in Mexico?',
  'answer': 'Uber operates in Mexico including ride sharing and delivery services. Lyft only operates in the United States and select cities in Canada.'},
 {'question': "Think step by step. First, consider the largest spanish speaking north american country that lyft operates in. Second describe any recent regulation that impacts Lyft's business in that market.",
  'answer': 'Lyft only operates in canada and the united states.'}]

In [6]:
eval_df = pd.DataFrame(eval_data)
eval_df.head()

,question,answer
0,What countries does Uber operate in?,Our technology is available in approximately 7...
1,What countries does Lyft operate in?,United states and select cities in Canada
2,Do Uber and Lyft operate in Mexico?,Uber operates in Mexico including ride sharing...
3,"Think step by step. First, consider the larges...",Lyft only operates in canada and the united st...
4,"Think step by step. First, consider the larges...","Uber operates in mexico. Since April 2019, Mex..."


# Eval Criteria

* Accurate information - Facts can be sourced to a ground truth
* Correct context - Extrapolation of the model is still in the context of the question
* Just enough information - No run on thoughts, just answer the question

Accurate
* Response is factually accurate, doesn't extend the answer beyond the scope of the question

Mixed
* Response contains the correct information, but either the context extended beyond the question or additional facts are added that have nothing to do with the question

Incorrect
* Any fact is not accurate, response may not include corret context

Successful Failure
* Model recognizes that it does not know the answer, interesting to see how well the model catches itself. This is helpful for UX purposes, and it would be good to have incorrect responses be this category

# Base Llama Performance

In [16]:
generation_pipeline = GenPipeline(model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
llama_3_1_responses = await save_answers_to_csv(answers, path="responses/base_llama_3_1_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/1zctM75Pd3dprofVs2Caky3kAv1LnyU1Z2hnxhbFeyPI/edit?usp=sharing

Correct: 3 (0.15)

Mixed: 7 (0.35)

Incorrect: 9 (0.45)

Successfull Failure: 1 (0.05)

In [17]:
generation_pipeline = GenPipeline(model_name = "meta-llama/Llama-3.2-3B-Instruct")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
llama_3_2_responses = await save_answers_to_csv(answers, path="responses/base_llama_3_2_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/1WrILF9GV6RS09LTQ2fHUHtH6SJs5v2DN61Ia55fIxME/edit?usp=sharing

Correct: 2 (0.1)

Mixed: 4 (0.2)

Incorrect: 13 (0.65)

Successfull Failure: 1 (0.05)

In [7]:
generation_pipeline = GenPipeline(model_name = "google/gemma-2-9b-it")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
gemma_2_responses = await save_answers_to_csv(answers, path="responses/base_gemma_2_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

Error in process_generation_batch, type: <class 'lamini.error.error.APIError'>, message: API error 524
Traceback (most recent call last):
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/generation/process_generation_batch.py", line 42, in process_generation_batch
    result = await query_api(client, key, url, json, batch["type"])
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/generation/process_generation_batch.py", line 85, in query_api
    result = await pipeline_client.completions(client, key, url, json)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/api/pipeline_client.py", line 17, in completions
    result = await mak

https://docs.google.com/spreadsheets/d/1W3TOvj-2s3zvr8nIInPSf6lTzyjwjmexzEYtCKJBHb8/edit?usp=sharing

Correct: 8 (0.4)

Mixed: 6 (0.3)

Incorrect: 3 (0.15)

Successfull Failure: 3 (0.15)

In [8]:
generation_pipeline = GenPipeline(model_name = "mistralai/Mistral-7B-Instruct-v0.3")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
mistral_responses = await save_answers_to_csv(answers, path="responses/base_mistral_7B_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

# RAG Performance

In [7]:
from lamini.index.lamini_index import LaminiIndex

In [9]:
llm_index = LaminiIndex.load_index("model_index")

In [10]:
embed = llm_index.get_embeddings("Ask three separate questions around a fact, table, or number within this text:")

In [ ]:
_, indices = llm_index.index.search(embed, 2)

In [ ]:
[llm_index.splits[i] for i in indices[0]]

In [5]:
llm_rag = RAGPipeline(rag_model_path="model_index", model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct")
answers = llm_rag.call(simple_prompt_generator(eval_df))
rag_3_1_responses = await save_answers_to_csv(answers, path="responses/rag_llama_3_1_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/17cl6vp8cEaAo3MmATy3L9zJyW37jHf4AQSfXaKKKr5s/edit?usp=sharing

Correct: 3 (0.15)

Mixed: 2 (0.1)

Incorrect: 1 (0.05)

Successfull Failure: 14 (0.7)

In [6]:
llm_rag = RAGPipeline(rag_model_path="model_index", model_name = "meta-llama/Llama-3.2-3B-Instruct")
answers = llm_rag.call(simple_prompt_generator(eval_df))
rag_3_2_responses = await save_answers_to_csv(answers, path="responses/rag_llama_3_2_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/1y9341YAzY0NDFIuF54PWpb-j3IW1gHMGe1afGwo7tjU/edit?usp=sharing

Correct: 4 (0.2)

Mixed: 1 (0.05)

Incorrect: 1 (0.05)

Successfull Failure: 14 (0.7)

In [7]:
llm_rag = RAGPipeline(rag_model_path="model_index", model_name = "google/gemma-2-9b-it", rag_query_size=1)
answers = llm_rag.call(simple_prompt_generator(eval_df))
rag_gemma_2_responses = await save_answers_to_csv(answers, path="responses/rag_gemma_2_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [9]:
llm_rag = RAGPipeline(rag_model_path="model_index", model_name = "mistralai/Mistral-7B-Instruct-v0.3", rag_query_size=1)
answers = llm_rag.call(simple_prompt_generator(eval_df))
rag_mistral_responses = await save_answers_to_csv(answers, path="responses/rag_mistral_7B_response.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

# Tuning Experiments

## Overfitting

In [28]:
tune_data = []
for idx, row in eval_df.iterrows():
    new_item = {
        "input": row["question"],
        "output": row["answer"]
    }
    tune_data.append(new_item)

In [29]:
base_llm = lamini.Lamini(model_name="meta-llama/Llama-3.2-3B-Instruct")
base_llm.tune(tune_data, finetune_args={"max_steps": 500, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: 7be6259d864fd673649461e0578126a8bf158a6ae29516d92503b35777b72577 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='7be6259d864fd673649461e0578126a8bf158a6ae29516d92503b35777b72577')
Tuning job submitted! Check status of job 13812 here: https://api.lamini.ai/train/13812


{'job_id': 13812,
 'status': 'CREATED',
 'dataset_id': '7be6259d864fd673649461e0578126a8bf158a6ae29516d92503b35777b72577'}

In [32]:
generation_pipeline = GenPipeline(model_name = "2d8f64af8f9c3ef80832ca2aadfdb7d0043fabfd819961293e169f768ba31225")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13812.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [30]:
base_llm = lamini.Lamini(model_name="google/gemma-2-9b-it")
base_llm.tune(tune_data, finetune_args={"max_steps": 500, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: 95e9cd9f82df5a7fd3e2fc979097a3e44dba0d941974ee3e44bcda58c3334cde . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='95e9cd9f82df5a7fd3e2fc979097a3e44dba0d941974ee3e44bcda58c3334cde')
Tuning job submitted! Check status of job 13813 here: https://api.lamini.ai/train/13813


{'job_id': 13813,
 'status': 'CREATED',
 'dataset_id': '95e9cd9f82df5a7fd3e2fc979097a3e44dba0d941974ee3e44bcda58c3334cde'}

In [35]:
generation_pipeline = GenPipeline(model_name = "e5efa53e755a7d0d6aba230292dd862c8be2634408f8b17ded5db404228ca448")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13813.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [31]:
base_llm = lamini.Lamini(model_name="mistralai/Mistral-7B-Instruct-v0.3")
base_llm.tune(tune_data, finetune_args={"max_steps": 500, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: e42a7c62960acf57811e214d52bfe38934d7d598d081ccb1d42544d37d671672 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='e42a7c62960acf57811e214d52bfe38934d7d598d081ccb1d42544d37d671672')
Tuning job submitted! Check status of job 13814 here: https://api.lamini.ai/train/13814


{'job_id': 13814,
 'status': 'CREATED',
 'dataset_id': 'e42a7c62960acf57811e214d52bfe38934d7d598d081ccb1d42544d37d671672'}

In [34]:
generation_pipeline = GenPipeline(model_name = "5997c763d14769bf00a89818f8dcaf51d4d67b30b5a9e5c9ae24bfd977605ea9")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13814.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

## Automatic QA Experiments

### V1

Job ID: 13265

Model ID: d9341ae5cfc60d5ed9e173954387404b6419ee201aa2baacb79fe4a6cef532a1

This is from an initial overfit run in the Contracts Data Discovery notebook, this was to overfit to a set of automatically extracted QA pairs

In [6]:
generation_pipeline = GenPipeline(model_name = "d9341ae5cfc60d5ed9e173954387404b6419ee201aa2baacb79fe4a6cef532a1")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13265.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

### Eval

Correct: 5 (0.25)

Mixed: 7 (0.35)

Incorrect: 6 (0.3)

Successfull Failure: 2 (0.1)

### V2

#### Generate QA Pairs

In [7]:
qa_pipeline = QuestionAnswerPipeline(
    question_system_prompt = "Ask three separate questions around a fact, table, or number within this text: \n"
)

##### Uber

In [8]:
answers = qa_pipeline.call(load_qa_prompts([uber_text[page] for page in uber_text]))
await save_answers(answers, path="uber_generated_qa.jsonl", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

##### Lyft

In [9]:
answers = qa_pipeline.call(load_qa_prompts([lyft_text[page] for page in lyft_text]))
await save_answers(answers, path="lyft_generated_qa.jsonl", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [19]:
tune_data = []
with jsonlines.Reader(open("uber_generated_qa.jsonl", "rb")) as reader:
    for line in reader:
        line["input"] = line["question"]
        line["output"] = line["answer"]
        tune_data.append(line)
with jsonlines.Reader(open("lyft_generated_qa.jsonl", "rb")) as reader:
    for line in reader:
        line["input"] = line["question"]
        line["output"] = line["answer"]
        tune_data.append(line)
tune_df = pd.DataFrame(tune_data)

In [20]:
tune_df.head()

,prompt,question,answer,input,output
0,<|begin_of_text|><|start_header_id|>user<|end_...,What is the name of the company that is filing...,"UBER TECHNOLOGIES, INC",What is the name of the company that is filing...,"UBER TECHNOLOGIES, INC"
1,<|begin_of_text|><|start_header_id|>user<|end_...,What is the state of incorporation of the company,Delaware,What is the state of incorporation of the company,Delaware
2,<|begin_of_text|><|start_header_id|>user<|end_...,What is the name of the exchange on which the ...,New York Stock Exchange,What is the name of the exchange on which the ...,New York Stock Exchange
3,<|begin_of_text|><|start_header_id|>user<|end_...,What is the number of shares of the registrant...,"1,954,464,088",What is the number of shares of the registrant...,"1,954,464,088"
4,<|begin_of_text|><|start_header_id|>user<|end_...,What is the aggregate market value of the voti...,approximately $90.5 billion,What is the aggregate market value of the voti...,approximately $90.5 billion


In [21]:
base_llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct")
base_llm.tune(tune_data)

Data pairs uploaded to local.

Your dataset id is: f00cbe4f6103fc99b731b351f7d963ac779909ca5f13cc7063c113e55d665f52 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='f00cbe4f6103fc99b731b351f7d963ac779909ca5f13cc7063c113e55d665f52')
Tuning job submitted! Check status of job 13695 here: https://api.lamini.ai/train/13695


{'job_id': 13695,
 'status': 'CREATED',
 'dataset_id': 'f00cbe4f6103fc99b731b351f7d963ac779909ca5f13cc7063c113e55d665f52'}

In [23]:
generation_pipeline = GenPipeline(model_name = "6bbfc54db98cb502a629aa5c3ed0aba4372fd2ad795ea694042ba6105ba892b4")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13695.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

##### Eval

Job ID: 13695

Model ID: 6bbfc54db98cb502a629aa5c3ed0aba4372fd2ad795ea694042ba6105ba892b4

Correct: 5 (0.25)

Mixed: 4 (0.2)

Incorrect: 8 (0.4)

Successfull Failure: 3 (0.15)

### V3

In [8]:
sum_pipeline = SummaryPipeline()

In [8]:
summaries = sum_pipeline.call(load_format_pages([uber_text[page] for page in uber_text]))
await save_summaries(summaries, path="v3_uber_generated_qa.jsonl", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [9]:
summaries = sum_pipeline.call(load_format_pages([lyft_text[page] for page in lyft_text]))
await save_summaries(summaries, path="v3_lyft_generated_qa.jsonl", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

In [10]:
tune_data = []
with jsonlines.Reader(open("v3_uber_generated_qa.jsonl", "rb")) as reader:
    for line in reader:
        tune_data.append(line)
with jsonlines.Reader(open("v3_lyft_generated_qa.jsonl", "rb")) as reader:
    for line in reader:
        tune_data.append(line)
tune_df = pd.DataFrame(tune_data)

In [11]:
tune_df.head()

,summary,content,input,output
0,This is the annual report of Uber Technologies...,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...,This is the annual report of Uber Technologies...,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...
1,The document appears to be a form for a compan...,_______________ Delaware45-2647441 (State or o...,The document appears to be a form for a compan...,_______________ Delaware45-2647441 (State or o...
2,The registrant is a well-known seasoned issuer...,k mark whether the registrant is a well-known ...,The registrant is a well-known seasoned issuer...,k mark whether the registrant is a well-known ...
3,The registrant has submitted all required Inte...,whether the registrant has submitted electroni...,The registrant has submitted all required Inte...,whether the registrant has submitted electroni...
4,The company is a large accelerated filer,Large accelerated filer\n☒ Accelerated filer ☐...,The company is a large accelerated filer,Large accelerated filer\n☒ Accelerated filer ☐...


In [18]:
base_llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct")
base_llm.tune(tune_data, finetune_args={"max_steps": 2000, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: c9dae5ed7286c881653e6421b85058a6074e58650828b0e80d55176e6cee8c65 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='c9dae5ed7286c881653e6421b85058a6074e58650828b0e80d55176e6cee8c65')
Tuning job submitted! Check status of job 13731 here: https://api.lamini.ai/train/13731


{'job_id': 13731,
 'status': 'CREATED',
 'dataset_id': 'c9dae5ed7286c881653e6421b85058a6074e58650828b0e80d55176e6cee8c65'}

meta-llama/Meta-Llama-3.1-8B-Instruct

This is from a previous run with different finetune args, this was using 500 steps with a learning rate of 0.0001

In [22]:
generation_pipeline = GenPipeline(model_name = "93d151c534c78c9ab1e7a6be0ac9abe445a5aec4b179aa0aad1a9e38631cec34")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13722.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/1eOoOglkQRe_pRX6RoeZd72PoNIgCa0GhJuD5mnw29uo/edit?usp=sharing

Correct: 0 (0)

Mixed: 0 (0)

Incorrect: 20 (1)

Successfull Failure: 0 (0)

In [23]:
generation_pipeline = GenPipeline(model_name = "3e0b53da66bad689d0107b6bb4e5f03ef0048113045cac3fcaf09f4dbe189e27")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13731.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/11RneXqA3y8I7ZZh6X0bhdUseFbjW4kCf-p6jcJtCPZU/edit?usp=sharing

Correct: 0 (0)

Mixed: 0 (0)

Incorrect: 20 (1)

Successfull Failure: 0 (0)

In [19]:
base_gemma = lamini.Lamini(model_name="google/gemma-2-9b-it")
base_gemma.tune(tune_data, finetune_args={"max_steps": 2000, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: 8901f7abc884d983bdfff67590a55b4f56015f9aba3aa3a7f79e0c0b7f7b599b . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='8901f7abc884d983bdfff67590a55b4f56015f9aba3aa3a7f79e0c0b7f7b599b')
Tuning job submitted! Check status of job 13732 here: https://api.lamini.ai/train/13732


{'job_id': 13732,
 'status': 'CREATED',
 'dataset_id': '8901f7abc884d983bdfff67590a55b4f56015f9aba3aa3a7f79e0c0b7f7b599b'}

In [24]:
generation_pipeline = GenPipeline(model_name = "ed1a20134bbfeca4c4e9963dad3d56db0c81bed6168a633a8a693649f7ab26a1")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13723.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

Error in process_generation_batch, type: <class 'lamini.error.error.APIError'>, message: API error 524
Traceback (most recent call last):
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/generation/process_generation_batch.py", line 42, in process_generation_batch
    result = await query_api(client, key, url, json, batch["type"])
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/generation/process_generation_batch.py", line 85, in query_api
    result = await pipeline_client.completions(client, key, url, json)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/powerml/Library/Caches/pypoetry/virtualenvs/contract-poc-jPOtkveY-py3.12/lib/python3.12/site-packages/lamini/api/pipeline_client.py", line 17, in completions
    result = await mak

https://docs.google.com/spreadsheets/d/1SZrI3xBcKF1B4MFrB5Jz1FwzTEkuh0glqruYoG7ruwE/edit?usp=sharing

Correct: 10 (0.50)

Mixed: 7 (0.35)

Incorrect: 3 (0.15)

Successfull Failure: 0 (0)

In [25]:
generation_pipeline = GenPipeline(model_name = "0226291e7bba69c0bef0a9f28c8956a59a7212593bfa5807ead3115378ab3f48")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13732.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

https://docs.google.com/spreadsheets/d/1--eFNnJtpRTkiBagyOSQnyxXvb6XyZ59z31xDHDAk-s/edit?usp=sharing

Correct: 9 (0.45)

Mixed: 5 (0.25)

Incorrect: 6 (0.3)

Successfull Failure: 0 (0)

In [20]:
base_mistral = lamini.Lamini(model_name="mistralai/Mistral-7B-Instruct-v0.3")
base_mistral.tune(tune_data, finetune_args={"max_steps": 2000, "learning_rate":0.0001})

Data pairs uploaded to local.

Your dataset id is: 5492cf45eb13468b0067ffa0e175a6fe0a81bf83485dbed68386c7dd5e6b364d . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='5492cf45eb13468b0067ffa0e175a6fe0a81bf83485dbed68386c7dd5e6b364d')
Tuning job submitted! Check status of job 13733 here: https://api.lamini.ai/train/13733


{'job_id': 13733,
 'status': 'CREATED',
 'dataset_id': '5492cf45eb13468b0067ffa0e175a6fe0a81bf83485dbed68386c7dd5e6b364d'}

In [26]:
generation_pipeline = GenPipeline(model_name = "b19ff90ee72d63a4019210db7f717341ad2f43e64fb11fb46e6ce7f7e253ef6b")
answers = generation_pipeline.call(simple_prompt_generator(eval_df))
model_responses = await save_answers_to_csv(answers, path="responses/gold_test_responses_13733.csv", print_outputs=False)

Saving answers: 0 answers [00:00, ? answers/s]

### V4

Shifting to Mistral as it had good base performance